#Rede Neural para regressão - tf.keras.Sequential 

Neste exemplo vamos utilizar as bilbiotecas Numpy e Tensorflow para manipulação de tensores e Pandas para manipulação de dados estruturados

In [0]:
import tensorflow as tf
import numpy as np
import pandas as pd

longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity

Neste exemplo vamos simular como aplicar uma regressão em um grande volume de dados. Sendo assim, para carregar nossos dados, vamos utilizar uma função para gerar batches, processá-los durante a leitura e criar um objeto do tipo dataset.

A célula a seguir é apenas auxiliar. Em ambientes com grande volume de dados não é possível carregar tudo em memória. Nesta célula estão as informações que seriam extraídas previamente da fote dos dados.

As colunas com dados numéricos e categóricos são distintas para agilizar o pré-processamento

In [0]:
NUMERIC_COLUMNS = ['longitude', 'latitude', 'housing_median_age', 'total_rooms',
       'total_bedrooms', 'population', 'households', 'median_income']

CATEGORICAL_COLUMNS = ['ocean_proximity']
VALUE_COMLUMN = 'median_house_value'

CATEGORIES = {
    'ocean_proximity': ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
}

data_example = pd.read_csv('housing.csv')
desc = data_example[NUMERIC_COLUMNS].describe()
MEAN = np.array(desc.T['mean'])
STD = np.array(desc.T['std'])

In [4]:
data_example.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,NEAR BAY
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,NEAR BAY


## Modelo de Regressão

### Carregamento de dados

A função a seguir nos auxiliará a visulizar os batches carregados

In [0]:
def show_batch(dataset):
  for batch, label in dataset.take(1):
    for key, value in batch.items():
      print("{:20s}: {}".format(key,value.numpy()))

Vamos inciar criando um método para fazer a leitura dos dados de um csv e criar um objeto de dataset do tensorflow. Nela definimos o tamanho do batch, os valores padrões para dados ausentes e qual campo representa nossa variável dependente (Y)

In [0]:
def get_dataset(file_path, batch_size):
  dataset = tf.data.experimental.make_csv_dataset(
      file_path,
      batch_size=batch_size, 
      label_name=VALUE_COMLUMN,
      na_value='0',
      num_epochs=1,
      ignore_errors=True)
  return dataset

In [0]:
raw_train_data = get_dataset('housing.csv', 10)

Enquanto não houver requisição de leitura dos dados, e.g. *take(1)*, eles não são carregados em memória e o objeto permanece vazio, mantendo apenas a estrutura do dataset.

In [8]:
raw_train_data

<PrefetchDataset shapes: (OrderedDict([(longitude, (None,)), (latitude, (None,)), (housing_median_age, (None,)), (total_rooms, (None,)), (total_bedrooms, (None,)), (population, (None,)), (households, (None,)), (median_income, (None,)), (ocean_proximity, (None,))]), (None,)), types: (OrderedDict([(longitude, tf.float32), (latitude, tf.float32), (housing_median_age, tf.float32), (total_rooms, tf.float32), (total_bedrooms, tf.float32), (population, tf.float32), (households, tf.float32), (median_income, tf.float32), (ocean_proximity, tf.string)]), tf.float32)>

Com o método *show_dataset* podemos visualizar o batch. Apenas um batch é carregado por vez.

In [9]:
show_batch(raw_train_data)

longitude           : [-118.23 -117.98 -118.45 -119.81 -122.3  -119.82 -118.05 -118.96 -119.79
 -122.1 ]
latitude            : [34.13 33.92 34.02 36.83 38.3  36.78 34.04 35.38 36.79 37.63]
housing_median_age  : [47. 27. 41. 19. 21. 36. 33. 41. 19. 18.]
total_rooms         : [1162. 3700. 2956. 6789. 1108. 1582. 1348. 2417. 1524. 9963.]
total_bedrooms      : [ 235.    0.  700. 1200.  269.  313.    0.  435.  448. 2031.]
population          : [ 781. 1793. 1212. 2325.  524.  761. 1098.  973.  960. 5613.]
households          : [ 268.  552.  645. 1109.  274.  318.  257.  406.  386. 1946.]
median_income       : [4.6528 5.3668 3.4583 4.049  2.7619 2.6055 4.2917 3.0568 1.5122 3.8171]
ocean_proximity     : [b'<1H OCEAN' b'<1H OCEAN' b'<1H OCEAN' b'INLAND' b'NEAR BAY' b'INLAND'
 b'<1H OCEAN' b'INLAND' b'INLAND' b'NEAR BAY']


Vamos criar o método o objeto *PackNumericFeatures* para criar um tensor que contenha apenas as características numéricas

In [0]:
class PackNumericFeatures(object):
  def __init__(self, names):
    self.names = names

  def __call__(self, features, labels):
    numeric_features = [features.pop(name) for name in self.names]
    numeric_features = [tf.cast(feat, tf.float32) for feat in numeric_features]
    numeric_features = tf.stack(numeric_features, axis=-1)
    features['numeric'] = numeric_features

    return features, labels

In [0]:
train_data = raw_train_data.map(
    PackNumericFeatures(NUMERIC_COLUMNS))

In [14]:
show_batch(train_data)

ocean_proximity     : [b'<1H OCEAN' b'INLAND' b'<1H OCEAN' b'INLAND' b'INLAND' b'<1H OCEAN'
 b'<1H OCEAN' b'<1H OCEAN' b'<1H OCEAN' b'INLAND']
numeric             : [[-1.1829e+02  3.3990e+01  4.3000e+01  1.9020e+03  3.9800e+02  1.1530e+03
   3.6300e+02  1.9375e+00]
 [-1.1974e+02  3.6780e+01  2.7000e+01  4.0490e+03  9.4700e+02  2.2540e+03
   8.8200e+02  2.2467e+00]
 [-1.1837e+02  3.3870e+01  1.9000e+01  7.5700e+02  1.4800e+02  3.6100e+02
   1.4100e+02  6.0200e+00]
 [-1.1969e+02  3.6810e+01  1.5000e+01  2.8920e+03  4.9600e+02  1.6340e+03
   5.0100e+02  4.4934e+00]
 [-1.1969e+02  3.6800e+01  3.1000e+01  2.5760e+03  4.5800e+02  1.3060e+03
   4.1800e+02  3.2813e+00]
 [-1.1832e+02  3.3940e+01  3.7000e+01  2.7400e+03  5.0400e+02  1.4680e+03
   4.7900e+02  4.5368e+00]
 [-1.1817e+02  3.3920e+01  4.3000e+01  2.0990e+03  3.9800e+02  1.2760e+03
   3.8700e+02  3.1528e+00]
 [-1.1847e+02  3.4010e+01  4.4000e+01  2.1750e+03  4.7500e+02  1.0190e+03
   4.4800e+02  4.7930e+00]
 [-1.2402e+02  4.0720e+01  

Para obtermos um treino mais estável e melhor performance ao aplicar o gradiente descendente, recomenda-se normalizar os dados. Uma forma é mantes média 0 e desvio padrão 1

In [0]:
def normalize_numeric_data(X):
  return (X-MEAN)/STD

Um dos objetos que possibilitam a manipulação de dados entre a fonte crua e a entrada em modelos no tensorflow 

*tf.feature_column.numeric_column* nos possibilitará manipulações de dados numéricos

In [16]:
numeric_column = tf.feature_column.numeric_column('numeric', normalizer_fn=normalize_numeric_data, shape=[len(NUMERIC_COLUMNS)])
numeric_columns = [numeric_column]
numeric_column

NumericColumn(key='numeric', shape=(8,), default_value=None, dtype=tf.float32, normalizer_fn=<function normalize_numeric_data at 0x7fa2e59d9b70>)

In [0]:
example_batch, labels = next(iter(train_data))

In [18]:
example_batch

OrderedDict([('ocean_proximity', <tf.Tensor: shape=(10,), dtype=string, numpy=
              array([b'NEAR BAY', b'NEAR BAY', b'<1H OCEAN', b'NEAR BAY', b'NEAR BAY',
                     b'<1H OCEAN', b'NEAR BAY', b'<1H OCEAN', b'NEAR BAY', b'<1H OCEAN'],
                    dtype=object)>),
             ('numeric', <tf.Tensor: shape=(10, 8), dtype=float32, numpy=
              array([[-1.2203e+02,  3.7940e+01,  2.1000e+01,  5.5410e+03,  7.7600e+02,
                       2.2140e+03,  7.3700e+02,  5.5777e+00],
                     [-1.2258e+02,  3.7980e+01,  5.2000e+01,  3.6750e+03,  6.4900e+02,
                       1.5530e+03,  6.3900e+02,  4.6905e+00],
                     [-1.1802e+02,  3.3930e+01,  3.5000e+01,  2.4000e+03,  3.9800e+02,
                       1.2180e+03,  4.0800e+02,  4.1312e+00],
                     [-1.2209e+02,  3.7650e+01,  3.5000e+01,  1.1300e+03,  1.9200e+02,
                       5.4300e+02,  1.8400e+02,  4.3897e+00],
                     [-1.2231e+02,  3

A saída da camada *numeric_layer* será um tensor com os dados já normalizados


In [19]:
numeric_layer = tf.keras.layers.DenseFeatures(numeric_columns)
numeric_layer(example_batch).numpy()

array([[-1.6388040e+00,  1.3578739e+00, -6.4357263e-01,  1.2801857e+00,
         5.3061253e-01,  6.7473537e-01,  5.9384203e-01,  9.3965578e-01],
       [-1.9376200e+00,  1.3773947e+00,  1.8631470e+00,  4.5700023e-01,
         2.4051251e-01,  9.6183583e-02,  3.4522760e-01,  4.6987909e-01],
       [ 5.3982520e-01, -5.9903520e-01,  4.8849431e-01, -1.0546570e-01,
        -3.3283484e-01, -1.9703102e-01, -2.4079211e-01,  1.7372717e-01],
       [-1.6714005e+00,  1.2163532e+00,  4.8849431e-01, -6.6572589e-01,
        -8.0339080e-01, -7.8783655e-01, -8.0905366e-01,  3.1060418e-01],
       [-1.7909271e+00,  1.5335568e+00,  1.2971135e+00,  1.6937059e-01,
         2.6335502e-01,  3.0274668e-01,  3.9089146e-01, -1.0548057e-01],
       [ 3.6053565e-01, -4.3311256e-01,  2.4590854e-01, -8.9777064e-01,
        -9.9526805e-01, -9.8827279e-01, -9.8917228e-01,  1.2711257e+00],
       [-1.7528963e+00,  1.2993135e+00,  1.8631470e+00,  3.7141716e-01,
         4.7350624e-01,  7.7942823e-04,  5.4564124e-01,  5

Da mesma forma que pre-processamos os dados numéricos, é preciso realizar alguns procimentos nos dados categóricos. O principal deles é criar uma representação numérica para eles. Em nosso caso, vamos criar um one-hot-encoding

In [0]:
#  CATEGORIES: dicionário que mapeia a colunas numéricas a todos os valores que ela pode possuir
categorical_columns = []
for feature, vocab in CATEGORIES.items():
  cat_col = tf.feature_column.categorical_column_with_vocabulary_list(
        key=feature, vocabulary_list=vocab)
  categorical_columns.append(tf.feature_column.indicator_column(cat_col))

In [21]:
categorical_columns

[IndicatorColumn(categorical_column=VocabularyListCategoricalColumn(key='ocean_proximity', vocabulary_list=('<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN'), dtype=tf.string, default_value=-1, num_oov_buckets=0))]

Assim como a *numeric_layer* criada, a saída da *categorical_layer* será um tensor com os dados processados 

In [22]:
categorical_layer = tf.keras.layers.DenseFeatures(categorical_columns)
print(categorical_layer(example_batch).numpy())

[[0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


Agora criamos uma última camada de pré-processamento, que será a composição da numérica e categórica. A saída dessa camada, será a entrada para o modelo.

In [0]:
preprocessing_layer = tf.keras.layers.DenseFeatures(categorical_columns+numeric_columns)

In [24]:
print(preprocessing_layer(example_batch).numpy())

[[-1.6388040e+00  1.3578739e+00 -6.4357263e-01  1.2801857e+00
   5.3061253e-01  6.7473537e-01  5.9384203e-01  9.3965578e-01
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00]
 [-1.9376200e+00  1.3773947e+00  1.8631470e+00  4.5700023e-01
   2.4051251e-01  9.6183583e-02  3.4522760e-01  4.6987909e-01
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00]
 [ 5.3982520e-01 -5.9903520e-01  4.8849431e-01 -1.0546570e-01
  -3.3283484e-01 -1.9703102e-01 -2.4079211e-01  1.7372717e-01
   1.0000000e+00  0.0000000e+00  0.0000000e+00  0.0000000e+00
   0.0000000e+00]
 [-1.6714005e+00  1.2163532e+00  4.8849431e-01 -6.6572589e-01
  -8.0339080e-01 -7.8783655e-01 -8.0905366e-01  3.1060418e-01
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+00
   0.0000000e+00]
 [-1.7909271e+00  1.5335568e+00  1.2971135e+00  1.6937059e-01
   2.6335502e-01  3.0274668e-01  3.9089146e-01 -1.0548057e-01
   0.0000000e+00  0.0000000e+00  0.0000000e+00  1.0000000e+0

## Criando o modelo com tf.keras.Sequential

In [0]:
def create_model(preprocessing_layer):
  model = tf.keras.Sequential([
    preprocessing_layer,
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1),
  ])

  model.compile(
      loss=tf.keras.losses.MeanSquaredError(),
      optimizer='adam',
      metrics=['mse'])
  return model

In [26]:
model = create_model(preprocessing_layer)
model.fit(train_data, epochs=10)

Epoch 1/10
2064/2064 [==============================] - 3s 2ms/step - loss: 26174494720.0000 - mse: 26174494720.0000
Epoch 2/10
2064/2064 [==============================] - 3s 2ms/step - loss: 5653015040.0000 - mse: 5653015040.0000
Epoch 3/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4896101376.0000 - mse: 4896101376.0000
Epoch 4/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4718416896.0000 - mse: 4718416896.0000
Epoch 5/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4648616448.0000 - mse: 4648616448.0000
Epoch 6/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4594756608.0000 - mse: 4594756608.0000
Epoch 7/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4567065600.0000 - mse: 4567065600.0000
Epoch 8/10
2064/2064 [==============================] - 3s 2ms/step - loss: 4533727232.0000 - mse: 4533727232.0000
Epoch 9/10
2064/2064 [==============================] - 3s 2ms/step - loss: 45

In [0]:
pred = model.predict(train_data)

In [28]:
pred[0]

array([377961.66], dtype=float32)

In [29]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_features_2 (DenseFeatu multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  1792      
_________________________________________________________________
dense_1 (Dense)              multiple                  16512     
_________________________________________________________________
dense_2 (Dense)              multiple                  129       
Total params: 18,433
Trainable params: 18,433
Non-trainable params: 0
_________________________________________________________________
